In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report
import glob

# Load and preprocess the data
csv_files = glob.glob('C:/Users/abdulssekyanzi/EDA Dataset.csv/100.csv')

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'time_ms'], axis=1)

# Create dummy labels
num_samples = len(df)
labels = np.random.randint(0, 3, num_samples)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df[['MLII', 'V5']], labels, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train_one_hot = keras.utils.to_categorical(y_train)
y_test_one_hot = keras.utils.to_categorical(y_test)

# Dummy signal quality labels
signal_quality_labels_train = np.random.randint(0, 2, len(X_train))
signal_quality_labels_test = np.random.randint(0, 2, len(X_test))

# One-hot encode signal quality labels
signal_quality_train_one_hot = keras.utils.to_categorical(signal_quality_labels_train)
signal_quality_test_one_hot = keras.utils.to_categorical(signal_quality_labels_test)

# Define the model
input_layer = layers.Input(shape=(2,))
shared_layers = layers.Dense(128, activation='relu')(input_layer)
shared_layers = layers.Dropout(0.3)(shared_layers)
shared_layers = layers.Dense(64, activation='relu')(shared_layers)
shared_layers = layers.Dropout(0.3)(shared_layers)
shared_layers = layers.Dense(32, activation='relu')(shared_layers)
shared_layers = layers.Dropout(0.3)(shared_layers)

# Task 1: Arrhythmia type classification
arrhythmia_output = layers.Dense(y_train_one_hot.shape[1], activation='softmax', name='arrhythmia')(shared_layers)

# Task 2: Signal quality classification
signal_quality_output = layers.Dense(signal_quality_train_one_hot.shape[1], activation='softmax', name='signal_quality')(shared_layers)

model = keras.Model(inputs=input_layer, outputs=[arrhythmia_output, signal_quality_output])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss={'arrhythmia': 'categorical_crossentropy', 'signal_quality': 'categorical_crossentropy'},
              metrics={'arrhythmia': 'accuracy', 'signal_quality': 'accuracy'})

# Train the model
history = model.fit(X_train, {'arrhythmia': y_train_one_hot, 'signal_quality': signal_quality_train_one_hot},
                    epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
results = model.evaluate(X_test, {'arrhythmia': y_test_one_hot, 'signal_quality': signal_quality_test_one_hot})

print("Test Loss (Arrhythmia):", results[1])
print("Test Accuracy (Arrhythmia):", results[3])
print("Test Loss (Signal Quality):", results[2])
print("Test Accuracy (Signal Quality):", results[4])

# Predictions and evaluation for arrhythmia task
arrhythmia_predictions = model.predict(X_test)[0]
arrhythmia_predicted_labels = np.argmax(arrhythmia_predictions, axis=1)

print("Arrhythmia Classification Report:")
print(classification_report(y_test, arrhythmia_predicted_labels))

# Predictions and evaluation for signal quality task
signal_quality_predictions = model.predict(X_test)[1]
signal_quality_predicted_labels = np.argmax(signal_quality_predictions, axis=1)

print("Signal Quality Classification Report:")
print(classification_report(signal_quality_labels_test, signal_quality_predicted_labels))

if results[3] >= 0.8:
    print("Arrhythmia Accuracy Goal of 80%+ achieved")
else:
    print("Arrhythmia Accuracy Goal not reached. Consider adjusting model parameters.")


Epoch 1/100
13000/13000 ━━━━━━━━━━━━━━━━━━━━ 353s 25ms/step - arrhythmia_accuracy: 0.3325 - arrhythmia_loss: 1.0997 - loss: 1.7936 - signal_quality_accuracy: 0.5008 - signal_quality_loss: 0.6939 - val_arrhythmia_accuracy: 0.3355 - val_arrhythmia_loss: 1.0987 - val_loss: 1.7918 - val_signal_quality_accuracy: 0.4994 - val_signal_quality_loss: 0.6932
Epoch 2/100
13000/13000 ━━━━━━━━━━━━━━━━━━━━ 427s 29ms/step - arrhythmia_accuracy: 0.3348 - arrhythmia_loss: 1.0987 - loss: 1.7920 - signal_quality_accuracy: 0.4997 - signal_quality_loss: 0.6932 - val_arrhythmia_accuracy: 0.3331 - val_arrhythmia_loss: 1.0987 - val_loss: 1.7919 - val_signal_quality_accuracy: 0.4992 - val_signal_quality_loss: 0.6933
Epoch 3/100
13000/13000 ━━━━━━━━━━━━━━━━━━━━ 384s 29ms/step - arrhythmia_accuracy: 0.3336 - arrhythmia_loss: 1.0987 - loss: 1.7919 - signal_quality_accuracy: 0.4991 - signal_quality_loss: 0.6932 - val_arrhythmia_accuracy: 0.3314 - val_arrhythmia_loss: 1.0987 - val_loss: 1.7921 - val_signal_quality_a

C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     43152
           1       0.00      0.00      0.00     43284
           2       0.34      1.00      0.50     43564

    accuracy                           0.34    130000
   macro avg       0.11      0.33      0.17    130000
weighted avg       0.11      0.34      0.17    130000

4063/4063 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step
Signal Quality Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67     65183
           1       0.00      0.00      0.00     64817

    accuracy                           0.50    130000
   macro avg       0.25      0.50      0.33    130000
weighted avg       0.25      0.50      0.33    130000

Arrhythmia Accuracy Goal not reached. Consider adjusting model parameters.


C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\abdulssekyanzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi